In [20]:
study=((0.5,0),(0.75,0),(1.0,0),(1.25,0),(1.50,0),(1.75,0),(2.0,0),(2.25,1),(2.5,0),(2.75,1),(3.0,0),(3.25,1),(3.5,0),(4.0,1),(4.25,1),(4.5,1),(4.75,1),(5.0,1),(5.5,1)) 

In [19]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()

In [5]:

import numpy as np
a = np.array(study).astype('float32')

In [6]:
features = a[:,0]
labels = a[:,1]

print(features)
print(labels)


[0.5  0.75 1.   1.25 1.5  1.75 2.   2.25 2.5  2.75 3.   3.25 3.5  4.
 4.25 4.5  4.75 5.   5.5 ]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1.]


2

In [70]:
import boto3
 


sess = sagemaker.Session()
bucket = "sagemakerlr"
prefix = "sagemaker/grades"

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, a, labels)
buf.seek(0)

key = 'linearlearner'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))


output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))


uploaded training data location: s3://sagemakerlr/sagemaker/grades/train/linearlearner
training artifacts will be uploaded to: s3://sagemakerlr/sagemaker/grades/output


In [71]:

containers = {
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest'
              }


In [72]:
containers[boto3.Session().region_name]


'382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest'

In [73]:
linear = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                       role=role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.p2.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)

In [75]:
%%time
linear.set_hyperparameters(feature_dim=2,
                           mini_batch_size=4,
                           predictor_type='binary_classifier')

linear.fit({'train': s3_train_data})



INFO:sagemaker:Creating training-job with name: linear-learner-2018-04-07-14-33-25-761


................................................................
Docker entrypoint called with argument(s): train
[04/07/2018 14:38:43 INFO 140516772214592] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'10', u'init_bias': u'0.0', u'lr_scheduler_factor': u'0.99', u'num_calibration_samples': u'10000000', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'10', u'lr_scheduler_step': u'100', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'0.00001', u'target_recall': u'0.8', u'num_models': u'32', u'early_stopping_patience': u'3', u'momentum': u'0.0', u'unbias_label': u'auto', u'wd': u'0.0', u'optimizer': u'adam', u'early_stopping_tolerance': u'0.001', u'learning_rate': u'auto', u'_kvstore': u'auto', u'normalize_data': u'true', u'binary_classifier_model_selec

===== Job Complete =====
Billable seconds: 173
CPU times: user 344 ms, sys: 32 ms, total: 376 ms
Wall time: 6min 8s


In [76]:
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.p2.xlarge')

INFO:sagemaker:Creating model with name: linear-learner-2018-04-07-14-40-41-204
INFO:sagemaker:Creating endpoint with name linear-learner-2018-04-07-14-33-25-761


--------------------------------------------------------------------------!

In [77]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

In [81]:
a[0]

array([0.5, 0. ], dtype=float32)

In [82]:
result = linear_predictor.predict(a[0])
print(result)

{'predictions': [{'score': 0.004625876434147358, 'predicted_label': 0.0}]}


In [83]:
containers[boto3.Session().region_name]

'382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest'